In [233]:
from sympy import *
import numpy as np

In [234]:
"""
author : YorozuyaSaint
https://github.com/berylgithub
"""

def forward_diff_parabolic(u, range_x, range_t, h, k, alpha, log=True):
    """
    numerical solution using forward difference method for parabolic heat equation
    """

    x=Symbol('x')
    t=Symbol('t')
    if log:
        print("u(x,t) = ",u(x,t))
        print("h=",h, ";x=",range_x[1])
        print("k=",k, ";t=",range_t[1])
        print("alpha=",alpha)
        
    l = (alpha**2)*k/(h**2)
    max_iter_i = int((range_x[1]-range_x[0])/h) + 1
    max_iter_j = int((range_t[1]-range_t[0])/k) + 1
    m = max_iter_i-1
    #initiate tridiagonal matrix A 
    A = np.zeros((m-1, m-1))
    for i in range(m-1):
        for j in range(m-1):
            if i==0:
                A[i] = np.array( [1-(2*l), l]+([0]*(len(A)-2)) )
            elif i==len(A)-1:
                A[i] = np.array( ([0]*(len(A)-2))+[l, 1-(2*l)] )
            else:
                A[i][i-1] = l
                A[i][i] = (1-(2*l))
                A[i][i+1] = l
    #insert zeros for u(0,t) and u(l,t)
    #A=np.insert(A, 0, np.zeros(m-1), 0)
    #A=np.insert(A, len(A), np.zeros(m-1), 0)
    if log:
        print("A = ")
        print(A)

    #initiate vector x input
    xi=np.array([h*i for i in range(max_iter_i)])

    #initiate w with size (m+1) * (m+1) (2 rows for u(0,t) and u(l,t))
    #w = np.zeros((max_iter_i, max_iter_j))
    w = np.zeros((m-1, max_iter_j))
    
    #initiate vector w at j=0, where 1<=i<=m-1, where w(i,0) = f(xi) 
    w10 = np.array([u(x,t).evalf(subs={x:xi[i], t:0}) for i in range(1, m)])
    
    #add zeros at both 0 and l
    #w10=np.insert(w10, 0, 0, 0)
    #w10=np.insert(w10, len(w10), 0, 0)
    
    #set it as w(1), but transpose w first, then insert w(1,0)
    w = np.transpose(w)
    w[0] = w10
    #w = np.transpose(w)
    #print(np.transpose(w))
    
    #calculate the w at j>0 until j*k = t
    for j in range(1, len(w)):
        w[j] = np.matmul(A, np.transpose(w[j-1]))
    if log :
        print("w(",len(w)-1,") = ")
        print(w[-1])
    return w[-1]


def backward_diff_parabolic(u, l, T, m, N, alpha, log=True):
    """
    backward difference solution to parabolic heat partial differential equation
    """
    x=Symbol('x')
    t=Symbol('t')
    if log:
        print("u(x,t) = ",u(x,t))
        print("l = ",l)
        print("T = ",T)
        print("m = ",m)
        print("N = ",N)
        print("alpha = ",alpha)
        
    #step 1
    h=l/m
    k=T/N
    var_lambda = (alpha**2)*k/(h**2)
    
    #step 2
    w = np.zeros(m)
    for i in range(1, m):
        w[i] = u(x,t).evalf(subs={x:(i*h), t:0})
    
    #step 3
    l = np.zeros(m)
    u = np.zeros(m)
    l[1] = 1+(2*var_lambda)
    u[1] = -var_lambda/l[1]
    
    #step 4
    for i in range(2, m-1):
        l[i] = 1 + (2*var_lambda) + (var_lambda*(u[i-1]))
        u[i] = -var_lambda/l[i]
        
    #step 5
    l[m-1] = 1 + (2*var_lambda) + (var_lambda*(u[m-2]))
    
    #step 6
    if log :
        print("Backward diff :")
    z = np.zeros(m)
    for j in range(1, N+1):
        #step 7
        t = j*k
        z[1] = w[1]/l[1]
        #step 8
        for i in range(2, m):
            z[i] = (w[i] + var_lambda*z[i-1])/l[i]
        #step 9
        w[m-1] = z[m-1]
        #step 10
        for i in range(m-2, 0, -1):
            w[i] = z[i] - (u[i]*w[i+1])
        #step 11
        if log:
            print("j iter = ",j)
            print("x\tw[i]")
        for i in range(1, m):
            x = i*h
            
            if log:
                print(x,"\t",w[i])
        if log:
            print()
    
    return w

def crank_nicolson_parabolic(u, l, T, m, N, alpha, log=True):
    """
    solves parabolic partial differential equation using crank-nicolson algorithm
    """
    x=Symbol('x')
    t=Symbol('t')
    
    if log:
        print("u(x,t) = ",u(x,t))
        print("l = ",l)
        print("T = ",T)
        print("m = ",m)
        print("N = ",N)
        print("alpha = ",alpha)
        
    #step 1
    h=l/m
    k=T/N
    var_lambda = (alpha**2)*k/(h**2)
    w = np.zeros(m+1)
    
    #step 2
    for i in range(1, m):
        w[i] = u(x,t).evalf(subs={x:(i*h), t:0})
    
    #step 3
    l = np.zeros(m)
    u = np.zeros(m)
    l[1] = 1+var_lambda
    u[1] = -var_lambda/(2*l[1])
    
    #step 4
    for i in range(2, m-1):
        l[i] = 1 + var_lambda + (var_lambda*(u[i-1])/2)
        u[i] = -var_lambda/(2*l[i])
        
    #step 5
    l[m-1] = 1 + var_lambda + (var_lambda*(u[m-2])/2)
    
    #step 6
    if log :
        print("Crank-Nicolson Algorithm :")
    z = np.zeros(m)
    for j in range(1, N+1):
        #step 7
        t = j*k
        z[1] = ( ((1-var_lambda)*w[1]) + (var_lambda*w[2]/2) )/l[1]
        #step 8
        for i in range(2, m):
            z[i] = (((1-var_lambda)*w[i]) + (var_lambda*(w[i+1] + w[i-1] + z[i-1])/2) )/l[i]
        #step 9
        w[m-1] = z[m-1]
        #step 10
        for i in range(m-2, 0, -1):
            w[i] = z[i] - (u[i]*w[i+1])
        #step 11
        if log:
            print("j iter = ",j)
            print("x\tw[i]")
        for i in range(1, m):
            x = i*h
            
            if log:
                print(x,"\t",w[i])
        if log:
            print()
    
    return w


In [235]:
#Example 1 page 727
print("Example 1 page 727")
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)

#variables initialization
alpha = 1
k = 0.0005
h = 0.1
rn_x = [0,1]
rn_t = [0,0.5]
x_iter = int((rn_x[1]-rn_x[0])/h)
xi = np.array([h*i for i in range(1, x_iter)])

#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:rn_t[1]}) for i in range(len(xi))])

#approximation of u(x,t)
w_last = forward_diff_parabolic(u, rn_x, rn_t, h, k, alpha)

print("\nResult :")
print("x\tu(x,0.5)\tw(i,1000)\terror")
for i in range(len(w_last)):
    err = abs(u_exact[i] - w_last[i])
    print(xi[i],"\t",u_exact[i],"\t",w_last[i],"\t",err)

Example 1 page 727
u(x,t) =  exp(-pi**2*t)*sin(pi*x)
h= 0.1 ;x= 1
k= 0.0005 ;t= 0.5
alpha= 1
A = 
[[0.9  0.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.05 0.9  0.05 0.   0.   0.   0.   0.   0.  ]
 [0.   0.05 0.9  0.05 0.   0.   0.   0.   0.  ]
 [0.   0.   0.05 0.9  0.05 0.   0.   0.   0.  ]
 [0.   0.   0.   0.05 0.9  0.05 0.   0.   0.  ]
 [0.   0.   0.   0.   0.05 0.9  0.05 0.   0.  ]
 [0.   0.   0.   0.   0.   0.05 0.9  0.05 0.  ]
 [0.   0.   0.   0.   0.   0.   0.05 0.9  0.05]
 [0.   0.   0.   0.   0.   0.   0.   0.05 0.9 ]]
w( 1000 ) = 
[0.00228652 0.00434922 0.00598619 0.00703719 0.00739934 0.00703719
 0.00598619 0.00434922 0.00228652]

Result :
x	u(x,0.5)	w(i,1000)	error
0.1 	 0.00222241417851267 	 0.0022865207865784528 	 6.41066080657787e-5
0.2 	 0.00422728297276244 	 0.004349220987439515 	 0.000121938014677076
0.30000000000000004 	 0.00581835585642586 	 0.005986189135245531 	 0.000167833278819674
0.4 	 0.00683988752999332 	 0.007037187382261514 	 0.000197299852268192
0.5 	 0.00719

In [236]:
#Example 2 page 731
print("Example 1 page 731")
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)

#variables initialization
l = 1
T = 0.5
m = 10
N = 50
alpha = 1
x_iter = int(m)
xi = np.array([h*i for i in range(1, x_iter)])

#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])

#approximation of u(x,t) using backward difference
w_last = backward_diff_parabolic(u, l, T, m, N, alpha)
w_last = w_last[1:]
print(w_last)
print("\nResult :")
print("x\tu(x,0.5)\tw(i,50)\terror")
for i in range(len(w_last)):
    err = abs(u_exact[i] - w_last[i])
    print(xi[i],"\t",u_exact[i],"\t",w_last[i],"\t",err)

Example 1 page 731
u(x,t) =  exp(-pi**2*t)*sin(pi*x)
l =  1
T =  0.5
m =  10
N =  50
alpha =  1
Backward diff :
j iter =  1
x	w[i]
0.1 	 0.28146521777558653
0.2 	 0.5353786589518121
0.30000000000000004 	 0.7368855067873765
0.4 	 0.8662608670353701
0.5 	 0.91084057802358
0.6000000000000001 	 0.86626086703537
0.7000000000000001 	 0.7368855067873764
0.8 	 0.5353786589518119
0.9 	 0.2814652177755864

j iter =  2
x	w[i]
0.1 	 0.256369941652248
0.2 	 0.48764460718115765
0.30000000000000004 	 0.6711852209394128
0.4 	 0.7890255488497042
0.5 	 0.8296305585743294
0.6000000000000001 	 0.7890255488497039
0.7000000000000001 	 0.6711852209394126
0.8 	 0.48764460718115743
0.9 	 0.25636994165224797

j iter =  3
x	w[i]
0.1 	 0.23351214584240515
0.2 	 0.44416649587496737
0.30000000000000004 	 0.6113427346013391
0.4 	 0.7186764869896372
0.5 	 0.7556611775178678
0.6000000000000001 	 0.7186764869896367
0.7000000000000001 	 0.6113427346013387
0.8 	 0.44416649587496704
0.9 	 0.23351214584240504

j iter =  4


In [ ]:
#Example 3 page 735
print("Example 3 page 735")
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)

#variables initialization
l = 1
T = 0.5
m = 10
N = 50
alpha = 1
x_iter = int(m)
xi = np.array([h*i for i in range(1, x_iter)])

#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])

#approximation of u(x,t) using backward difference
w_last = crank_nicolson_parabolic(u, l, T, m, N, alpha)
w_last = w_last[1:-1]
print(w_last)
print("\nResult :")
print("x\tu(x,0.5)\tw(i,50)\terror")
for i in range(len(w_last)):
    err = abs(u_exact[i] - w_last[i])
    print(xi[i],"\t",u_exact[i],"\t",w_last[i],"\t",err)

Example 3 page 735
u(x,t) =  exp(-pi**2*t)*sin(pi*x)
l =  1
T =  0.5
m =  10
N =  50
alpha =  1
Crank-Nicolson Algorithm :
j iter =  1
x	w[i]
0.1 	 0.2801796576381923
0.2 	 0.532933378260296
0.30000000000000004 	 0.7335198666530965
0.4 	 0.8623043197644631
0.5 	 0.9066804180298086
0.6000000000000001 	 0.8623043197644632
0.7000000000000001 	 0.7335198666530964
0.8 	 0.5329333782602959
0.9 	 0.28017965763819225

j iter =  2
x	w[i]
0.1 	 0.2540334091108448
0.2 	 0.4832002581830832
0.30000000000000004 	 0.6650680993301988
0.4 	 0.781834441112953
0.5 	 0.822069380438708
0.6000000000000001 	 0.781834441112953
0.7000000000000001 	 0.6650680993301987
0.8 	 0.48320025818308293
0.9 	 0.25403340911084465

j iter =  3
x	w[i]
0.1 	 0.2303271175661581
0.2 	 0.43810821208154915
0.30000000000000004 	 0.6030042223189949
0.4 	 0.7088739778983939
0.5 	 0.7453542095056734
0.6000000000000001 	 0.7088739778983938
0.7000000000000001 	 0.6030042223189946
0.8 	 0.438108212081549
0.9 	 0.23032711756615798

j it